In [ ]:
from typing import List, Dict

# from oop_functions.experiment_helper import *
# from oop_functions.experiment_runner import ExperimentRunner
# from oop_functions.util_functions import *
# from oop_functions.visualization_util import *
from oop_functions.analytics_cv_util import *
from oop_functions.analytics_utils import *

%matplotlib inline
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns

from xgboost import XGBClassifier

import itertools


In [ ]:
def convert_mapping_to_dict(my_mapping):
    name_to_choices_mapping = {}
    for mapping in my_mapping:
        for name in mapping["name"]:
            name_to_choices_mapping[name] = mapping["choices"]
    return name_to_choices_mapping

def survey_results_parser(survey_result, my_mapping):

    true_false_mapping = {
        "True": 1,
        "False": 0,
    }
    parsed_survey_result = {}
    name_to_choices_mapping = convert_mapping_to_dict(my_mapping)
    for key, val in survey_result.items():
        if key in name_to_choices_mapping:
            parsed_survey_result[key] = name_to_choices_mapping[key][val]
        elif val in ["True", "False"]:
            parsed_survey_result[key] = true_false_mapping[val]
        else:
            parsed_survey_result[key] = val
    return parsed_survey_result

In [ ]:
my_mapping = [
    {
        "name": ["ca125_result", "ovar_result", "tvu_result"],
        "choices": {
            "Negative": 1,
            "Abnormal, Suspicious": 2,
            "Abnormal, non-suspicious": 3,
            "Inadequate Screen": 4,
            "Not Done, Expected": 8,
            "Not Done, Not Expected": 9,
        },
    },
    {
        "name": ["tvu_ref"],
        "choices": {
            "Significant Abnormality, Referral": 1,
            "Moderate Abnormality, Referral": 2,
            "Slight Variation from Normal, No Referral": 3,
            "Normal / Result Not Available, No Referral": 4,
        }
    },
    {
        "name": ["ca125ii_level_binary"],
        "choices": {
            "True": 2,
            "False": 1,
        }
    },
    {
        "name": ["ovcyst_morph"],
        "choices": {
            "Not visualized": 0,
            "No solid area and outline smooth": 1,
            "Mixed solid area or outline irregular or outline papillaries": 2,
            "Solid area": 3,
        },
    },
    ]

In [ ]:
survey_result = {
  "ca125ii_level": 55,
  "ca125ii_level_binary": "False",
  "ca125_result": "Not Done, Not Expected",
  "ovar_result": "Abnormal, Suspicious",
  "tvu_ref": "Slight Variation from Normal, No Referral",
  "ovcyst_vol": 0,
  "ovary_vol": 2,
  "ovcyst_sum": 0,
  "detl_p": "False",
  "ovcyst_diam": 1,
  "ovary_volr": 2,
  "detr_p": "False",
  "ovcyst_volr": 0,
  "ovary_diam": 1,
  "ovcyst_diamr": 2,
  "bmi_curr": 32,
  "rantero_p": 12,
  "ovcyst_morph": "Not visualized",
  "tvu_result": "Not Done, Not Expected",
  "bmi_50": 25,
  "weight_f": 170,
  "weight50_f": 147,
  "bmi_20": 21,
  "ovary_voll": 1,
  "ovcyst_diaml": 0,
  "ovcyst_voll": 0,
  "lantero_p": 11,
  "weight20_f": 124,
  "volum": 8,
  "visl": "False",
  "ovary_diamr": 1,
  "rlong_p": 16,
  "viseith": "False",
  "rtran_p": 13,
  "maxdi": 0
}

In [ ]:
parsed_survey_result = survey_results_parser(survey_result, my_mapping)
X_predict = pd.DataFrame(parsed_survey_result, index=[0])
cols = ['bmi_20', 'bmi_50', 'bmi_curr', 'ca125_result', 'ca125ii_level',
       'ca125ii_level_binary', 'detl_p', 'detr_p', 'lantero_p', 'maxdi',
       'ovar_result', 'ovary_diam', 'ovary_diamr', 'ovary_vol', 'ovary_voll',
       'ovary_volr', 'ovcyst_diam', 'ovcyst_diaml', 'ovcyst_diamr',
       'ovcyst_morph', 'ovcyst_sum', 'ovcyst_vol', 'ovcyst_voll',
       'ovcyst_volr', 'rantero_p', 'rlong_p', 'rtran_p', 'tvu_ref',
       'tvu_result', 'viseith', 'visl', 'volum', 'weight20_f', 'weight50_f',
       'weight_f']
X_predict = X_predict[cols]

In [ ]:
filesuffix = '_for_experiment_participants_screened_single_first_5_top_35_features_RandomForestClassifier_cancer_in_next_1_years__1_trials'
cv_analytics_util = load_cv_analytics_utils_no_datasets(filesuffix)

In [ ]:
# cv_analytics_util = CvAnalyticsUtil.load_cv_analytics_utils(filesuffix)
# full_dataset = cv_analytics_util.get_dataset_with_predictions()
# cols = cv_analytics_util.analytics_utils[0].data_util.get_test_data()[0].columns

In [ ]:
# Predicting actual cancer probability using bucketing
per_thereshold_metrics = pd.read_csv('./per_threshold_metrics/_for_experiment_participants_screened_single_first_5_RandomForestClassifier_cancer_in_next_1_years__15_trials.csv')

In [ ]:
pred, prob = cv_analytics_util.analytics_utils[0].get_predictions_general(X_predict)

In [ ]:
fraction = map_prob_to_bucket(per_thereshold_metrics, prob[0])

In [ ]:
prob[0]

0.0033333333333333335

In [ ]:
fraction

0.0001589924029282